In [1]:
import pandas as pd

df = pd.read_json("hf://datasets/hbattu/huberman-youtube-metadata/metadata.jsonl", lines=True)

In [3]:
df.head()

,url,published,video_id,title,length,keywords,description,thumbnail,file_name
0,https://www.youtube.com/watch?v=eIxVfln02Ss,05/08/2023,eIxVfln02Ss,"How Psilocybin Can Rewire Our Brain, Its Thera...",7741,"[psilocybin, psychedelics]","In this episode, I discuss what psilocybin is ...",https://i.ytimg.com/vi/eIxVfln02Ss/default.jpg,mp3/eIxVfln02Ss.mp3
1,https://www.youtube.com/watch?v=cS7cNaBrkxo,05/01/2023,cS7cNaBrkxo,Dr. Noam Sobel: How Smells Influence Our Hormo...,11596,"[noam sobel, Weizmann Institute of Science, sm...","In this episode, my guest is Noam Sobel, PhD, ...",https://i.ytimg.com/vi/cS7cNaBrkxo/default.jpg,mp3/cS7cNaBrkxo.mp3
2,https://www.youtube.com/watch?v=0RYyQRQFgFk,04/24/2023,0RYyQRQFgFk,Science-Based Mental Training & Visualization ...,7187,"[mental training, visualization, improve learn...","In this episode, I explore the science of ment...",https://i.ytimg.com/vi/0RYyQRQFgFk/default.jpg,mp3/0RYyQRQFgFk.mp3
3,https://www.youtube.com/watch?v=3ZGItIAUQmI,04/17/2023,3ZGItIAUQmI,Dr. Matthew MacDougall: Neuralink & Technologi...,7301,"[matthew macdougall, neuralink, neurosurgeon, ...","In this episode, my guest is Matthew MacDougal...",https://i.ytimg.com/vi/3ZGItIAUQmI/default.jpg,mp3/3ZGItIAUQmI.mp3
4,https://www.youtube.com/watch?v=6ZrlsVx85ek,04/10/2023,6ZrlsVx85ek,"The Science of Healthy Hair, Hair Loss and How...",7487,"[hair growth, science based tools for hair gro...","In this episode, I explain the biology of hair...",https://i.ytimg.com/vi/6ZrlsVx85ek/default.jpg,mp3/6ZrlsVx85ek.mp3


In [5]:
!pip install youtube_transcript_api

   ---------------------------------------- 0.0/622.3 kB ? eta -:--:--
   ---------------------------------------- 622.3/622.3 kB 5.7 MB/s eta 0:00:00


In [7]:
from youtube_transcript_api import YouTubeTranscriptApi
def add_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        script = ""
        for elements in transcript:
            script = script + elements['text']
        return script
    except Exception as e:
        print(f"An error occurred: {e}")

In [9]:
df["script"] = df["video_id"].apply(add_transcript)

An error occurred: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=JPX8g8ibKFc! This is most likely caused by:

No transcripts were found for any of the requested language codes: ('en',)

For this video (JPX8g8ibKFc) transcripts are available in the following languages:

(MANUALLY CREATED)
 - en-US ("English (United States)")[TRANSLATABLE]
 - es ("Spanish")[TRANSLATABLE]

(GENERATED)
None

(TRANSLATION LANGUAGES)
 - ab ("Abkhazian")
 - aa ("Afar")
 - af ("Afrikaans")
 - ak ("Akan")
 - sq ("Albanian")
 - am ("Amharic")
 - ar ("Arabic")
 - hy ("Armenian")
 - as ("Assamese")
 - ay ("Aymara")
 - az ("Azerbaijani")
 - bn ("Bangla")
 - ba ("Bashkir")
 - eu ("Basque")
 - be ("Belarusian")
 - bho ("Bhojpuri")
 - bs ("Bosnian")
 - br ("Breton")
 - bg ("Bulgarian")
 - my ("Burmese")
 - ca ("Catalan")
 - ceb ("Cebuano")
 - zh-Hans ("Chinese (Simplified)")
 - zh-Hant ("Chinese (Traditional)")
 - co ("Corsican")
 - hr ("Croatian")
 - cs ("Czech")
 - da ("Danish")
 - d

In [11]:
df["script"]

0      ANDREW HUBERMAN: Welcome\nto the Huberman Lab ...
1      welcome to the huberman Lab podcastwhere we di...
2      welcome to the huberman Lab podcastwhere we di...
3      welcome to the huberman Lab podcastwhere we di...
4      ANDREW HUBERMAN: Welcome to the\nHuberman Lab ...
                             ...                        
124                                                 None
125                                                 None
126    [upbeat music]- Welcome to the Huberman Lab Po...
127    - Welcome to the Huberman Lab Podcastwhere we ...
128                                                 None
Name: script, Length: 129, dtype: object

In [13]:
df.dropna(inplace=True)

In [15]:
df.shape

(114, 10)

In [21]:
script1 = df.iloc[0]["script"]

In [33]:
def chunk_transcript_by_chars(transcript, max_chars=12000, overlap=2000):
    """
    Splits transcript by character length while ensuring slight overlap for context.
    """
    chunks = []
    start = 0
    while start < len(transcript):
        end = start + max_chars
        chunk = transcript[start:end]
        chunks.append(chunk.strip())
        start += max_chars - overlap
    return chunks


df["model_inputs"] = df["script"].apply(chunk_transcript_by_chars)

In [39]:
model_inputs = df["model_inputs"]

In [42]:
model_inputs

0      [ANDREW HUBERMAN: Welcome\nto the Huberman Lab...
1      [welcome to the huberman Lab podcastwhere we d...
2      [welcome to the huberman Lab podcastwhere we d...
3      [welcome to the huberman Lab podcastwhere we d...
4      [ANDREW HUBERMAN: Welcome to the\nHuberman Lab...
                             ...                        
109    [- Welcome to the Huberman Lab Podcastwhere we...
110    [- Welcome to the Huberman Lab Podcastwhere we...
119    [- Welcome to the Huberman Lab Podcastwhere we...
126    [[upbeat music]- Welcome to the Huberman Lab P...
127    [- Welcome to the Huberman Lab Podcastwhere we...
Name: model_inputs, Length: 114, dtype: object

In [50]:
final_data = []
for model_input in model_inputs:
    for i in model_input:
        final_input = f'''<s>[INST] HubermanGPT, functioning as a virtual neuroscience expert, communicates complex scientific concepts in an accessible manner.
        It escalates to deeper details on request and responds to feedback thoughtfully. HubermanGPT adapts the length of its responses based on the user's input, providing concise answers for brief comments or deeper explanations for detailed inquiries.[/INST]
        {i}</s>'''
        final_data.append(final_input)

In [46]:
print("<s>[INST] HubermanGPT, functioning as a virtual neuroscience expert, communicates complex scientific concepts in an accessible manner. It escalates to deeper details on request and responds to feedback thoughtfully. HubermanGPT adapts the length of its responses based on the user's input, providing concise answers for brief comments or deeper explanations for detailed inquiries. Please respond to the following comment: 'This is incredibly insightful, Huberman. I've been struggling with explaining the intricacies of neuroplasticity to my students. Can you simplify it for a general audience? I also think the description of dopamine release in your recent episode missed the role of the prefrontal cortex.' [/INST] Thanks for the feedback! Neuroplasticity can be a challenging topic, but it boils down to how our brain constantly adapts and reorganizes itself based on experiences, like learning new skills. The prefrontal cortex is key in regulating dopamine, and I’ll make sure to emphasize that in future episodes! I’ve also written a post that dives into this in more detail: [link to blog]. -HubermanGPT</s>")

<s>[INST] HubermanGPT, functioning as a virtual neuroscience expert, communicates complex scientific concepts in an accessible manner. It escalates to deeper details on request and responds to feedback thoughtfully. HubermanGPT adapts the length of its responses based on the user's input, providing concise answers for brief comments or deeper explanations for detailed inquiries. Please respond to the following comment: 'This is incredibly insightful, Huberman. I've been struggling with explaining the intricacies of neuroplasticity to my students. Can you simplify it for a general audience? I also think the description of dopamine release in your recent episode missed the role of the prefrontal cortex.' [/INST] Thanks for the feedback! Neuroplasticity can be a challenging topic, but it boils down to how our brain constantly adapts and reorganizes itself based on experiences, like learning new skills. The prefrontal cortex is key in regulating dopamine, and I’ll make sure to emphasize th

In [54]:
len(final_data)

1598

In [58]:
final_data_series = pd.Series(data=final_data)

In [66]:
final_data_series.name = "Transcript"

In [70]:
final_data_series.to_csv(r"C:\Users\vigne\Desktop\Higher studies\Northeastern University Boston\Courses\Semester 2\huberman_transcript.csv")

In [62]:
final_data_series.shape

(1598,)

In [72]:
final_data_series[0]

"<s>[INST] HubermanGPT, functioning as a virtual neuroscience expert, communicates complex scientific concepts in an accessible manner.\n        It escalates to deeper details on request and responds to feedback thoughtfully. HubermanGPT adapts the length of its responses based on the user's input, providing concise answers for brief comments or deeper explanations for detailed inquiries.[/INST]\n        ANDREW HUBERMAN: Welcome\nto the Huberman Lab podcastwhere we discuss science\nand science-based toolsfor everyday life.[MUSIC PLAYING]I'm Andrew Huberman,\nand I'm a professorof neurobiology\nand ophthalmologyat Stanford School of Medicine.Today, we are\ndiscussing psilocybin.Psilocybin is a psychedelic,\nmeaning it modifies the psyche.And in doing so, it changes\nour level of consciousness.Psychedelics, such as\npsilocybin changed the waythat we perceive the outside\nworld and our internal world,our memories, our thoughts,\nour feelings, et cetera.Not just while one is under\nthe inf